In [1]:
import sys

import pygame as pg
from pygame.locals import *
import numpy as np

from car import Car
from track import Track

import neat
import math
import time
import pickle

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
maps = [
    {'name': 'track01', 'file': 'tracks/track01.png', 'x_text': 1920 / 2, 'y_text': 1080 / 2, 'distance_text': 50},
    {'name': 'track02', 'file': 'tracks/track02.png', 'x_text': 1920 / 2, 'y_text': 1080 / 2, 'distance_text': 50},
    # {'name': 'track02', 'file': 'tracks/track02.png', 'x_text': 1716, 'y_text': 400, 'distance_text': 50}
]

In [3]:
class NeatCar(Car):
    MAX_LENGTH = 200
    Car.CAR_SIZE_X = 60
    Car.CAR_SIZE_Y = 60
    
    def get_reward(self):
        return self.distance / (Car.CAR_SIZE_X / 2) + self.has_touched_finish() * 10000
    
    def __init__(self, car_sprite, pos_x, pos_y, angle, speed, game_map, border_color, map_width, map_height, top_start_line, bottom_start_line):
        super().__init__(car_sprite, pos_x, pos_y, angle, speed, game_map, border_color, map_width, map_height, top_start_line, bottom_start_line)
        self.radars = []
        self.speed_changes = 0
        self.speed_incremental_average = 1
        self.angles = []
           
    def check_radar(self, degree):
        length = 0
        x = int(self.center[0] + length * math.cos(math.radians(360 - (self.angle + degree))))
        y = int(self.center[1] + length * math.sin(math.radians(360 - (self.angle + degree))))
        
        while not self.is_collision_points(x, y) and length < NeatCar.MAX_LENGTH:
            length += 1
            x = int(self.center[0] + length * math.cos(math.radians(360 - (self.angle + degree))))
            y = int(self.center[1] + length * math.sin(math.radians(360 - (self.angle + degree))))

        dist = int(pg.math.Vector2(x, y).distance_to(self.center))
        self.radars.append([(x, y), dist])
        
        if degree == 0:
            length = 0
            x = int(self.center[0] + length * math.cos(math.radians(360 - (self.angle + degree))))
            y = int(self.center[1] + length * math.sin(math.radians(360 - (self.angle + degree))))
            
            while not self.is_collision_points(x, y) and length < NeatCar.MAX_LENGTH + 100:
                length += 1
                x = int(self.center[0] + length * math.cos(math.radians(360 - (self.angle + degree))))
                y = int(self.center[1] + length * math.sin(math.radians(360 - (self.angle + degree))))
            
            dist = int(pg.math.Vector2(x, y).distance_to(self.center))
            self.radars.append([(x, y), dist])
            
        
    def draw(self, screen):
        super().draw(screen)        
        for r in self.radars:
            pg.draw.line(screen, (0, 255, 0), self.center, r[0], 1)
            pg.draw.circle(screen, (0, 255, 0), r[0], 5)

    def update(self):
        super().update()
        
        self.speed_incremental_average = (self.speed_incremental_average * self.speed_changes + self.speed) / (self.speed_changes + 1)
        
        self.radars.clear()
        for d in range(-90, 120, 30):
            self.check_radar(d)

    def get_neat_data(self):
        return [r[1] // 30 for r in self.radars] if len(self.radars) > 0 else [0, 0, 0, 0, 0, 0, 0, 0]
        

In [4]:
def run_simulation(genomes, config, speed = 20, full_screen = True):    
    networks = []
    cars = []

    pg.init()
    
    global current_map_path
    my_track = Track(current_map['file'])
    
    screen = pg.display.set_mode((my_track.map_width, my_track.map_height), (SCALED | RESIZABLE) if not full_screen else FULLSCREEN)
    my_track.load_game_map()
    top_start_line, bottom_start_line = my_track.get_start_line_points()

    clock = pg.time.Clock()

    start_pos_x, start_pos_y, angle = my_track.start_pos

    start_pos_y -= Car.CAR_SIZE_Y // 2

    for _, g in genomes:
        net = neat.nn.FeedForwardNetwork.create(g, config)
        networks.append(net)
        g.fitness = 0
        cars.append(
            NeatCar('cars/car2d.png', start_pos_x, start_pos_y, angle, speed, my_track.game_map, my_track.border_color,
                    my_track.width, my_track.height, top_start_line, bottom_start_line))

    font_generation = pg.font.SysFont("Arial", 30)
    font_alive = pg.font.SysFont("Arial", 20)

    global current_generation
    current_generation += 1

    timer = time.time()
    MAX_TIME = 60
    
    while True:
        for event in pg.event.get():
            if event.type == QUIT:
                pg.quit()
                
        no_still_alive = 0

        for i, car in enumerate(cars):
            output = networks[i].activate(car.get_neat_data())
            choice = output.index(max(output))

            car.move(choice)                
                
            if car.is_alive():
                no_still_alive += 1
                car.update()
                genomes[i][1].fitness = car.get_reward()

        if no_still_alive == 0 or time.time() - timer > MAX_TIME:
            break

        screen.blit(my_track.game_map, (0, 0))
        for car in cars:
            if car.is_alive():
                car.draw(screen)

        text_gen = font_generation.render("Generation: " + str(current_generation), True, (0, 0, 0))
        text_gen_rect = text_gen.get_rect()
        text_gen_rect.center = (current_map['x_text'], current_map['y_text'])
        screen.blit(text_gen, text_gen_rect)

        text_alive = font_alive.render("Alive: " + str(no_still_alive), True, (0, 0, 0))
        text_alive_rect = text_alive.get_rect()
        text_alive_rect.center = (current_map['x_text'], current_map['y_text'] + current_map['distance_text'])
        screen.blit(text_alive, text_alive_rect)

        pg.display.flip()
        clock.tick(60)

    # pg.quit()

max_fitness = []

In [5]:
current_map = maps[0]

current_generation = 0

config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, 'alg_neat/neat.config')
population = neat.Population(config)
population.add_reporter(neat.StdOutReporter(True))

stats = neat.StatisticsReporter()
population.add_reporter(stats)

population.run(run_simulation, 30)
pg.quit()

with open(f'alg_neat/best_genome_{current_map["name"]}.pkl', 'wb') as output:
    pickle.dump(population, output, 1)
    
best_genome = stats.best_genome()
max_fitness.append(best_genome.fitness)


 ****** Running generation 0 ****** 

Population's average fitness: 17.13222 stdev: 15.76406
Best fitness: 53.90000 - size: (5, 40) - species 1 - id 6
Average adjusted fitness: 0.293
Mean genetic distance 1.078, standard deviation 0.280
Population of 30 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    30     53.9    0.293     0
Total extinctions: 0
Generation time: 4.557 sec

 ****** Running generation 1 ****** 

Population's average fitness: 23.36111 stdev: 15.31581
Best fitness: 53.90000 - size: (5, 40) - species 1 - id 6
Average adjusted fitness: 0.413
Mean genetic distance 1.162, standard deviation 0.255
Population of 30 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1    30     53.9    0.413     1
Total extinctions: 0
Generation time: 4.740 sec (4.648 average)

 ****** Running generation 2 ****** 

Population's average fitness: 29.59667 stde

In [6]:
current_map = maps[1]

current_generation = 0

config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, 'alg_neat/neat.config')
population = neat.Population(config)
population.add_reporter(neat.StdOutReporter(True))

stats = neat.StatisticsReporter()
population.add_reporter(stats)

population.run(run_simulation, 60)
pg.quit()

with open(f'alg_neat/best_genome_{current_map["name"]}.pkl', 'wb') as output:
    pickle.dump(population, output, 1)
    
best_genome = stats.best_genome()
max_fitness.append(best_genome.fitness)


 ****** Running generation 0 ****** 

Population's average fitness: 15.24444 stdev: 15.70978
Best fitness: 39.83333 - size: (5, 40) - species 1 - id 10
Average adjusted fitness: 0.344
Mean genetic distance 1.224, standard deviation 0.197
Population of 30 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    30     39.8    0.344     0
Total extinctions: 0
Generation time: 4.635 sec

 ****** Running generation 1 ****** 

Population's average fitness: 34.85000 stdev: 11.16269
Best fitness: 39.83333 - size: (5, 40) - species 1 - id 10
Average adjusted fitness: 0.859
Mean genetic distance 1.043, standard deviation 0.324
Population of 30 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1    30     39.8    0.859     1
Total extinctions: 0
Generation time: 6.236 sec (5.436 average)

 ****** Running generation 2 ****** 

Population's average fitness: 28.14444 st

In [7]:
print(max_fitness)

[931.5, 3590.633333333333]


In [4]:
def run_simulation_competition(genomes, config, speed = 20, full_screen = True):    
    networks = []
    cars = []

    pg.init()
    
    global current_map_path
    my_track = Track(current_map['file'])
    
    screen = pg.display.set_mode((my_track.map_width, my_track.map_height), (SCALED | RESIZABLE) if not full_screen else FULLSCREEN)
    my_track.load_game_map()
    top_start_line, bottom_start_line = my_track.get_start_line_points()

    clock = pg.time.Clock()

    start_pos_x, start_pos_y, angle = my_track.start_pos

    start_pos_y -= Car.CAR_SIZE_Y // 2

    for _, g in genomes:
        net = neat.nn.FeedForwardNetwork.create(g, config)
        networks.append(net)
        g.fitness = 0
        cars.append(
            NeatCar('cars/car2d.png', start_pos_x, start_pos_y, angle, speed, my_track.game_map, my_track.border_color,
                    my_track.width, my_track.height, top_start_line, bottom_start_line))

    font_generation = pg.font.SysFont("Arial", 30)
    font_alive = pg.font.SysFont("Arial", 20)

    global current_generation
    current_generation += 1

    timer = time.time()
    MAX_TIME = 60
    
    avg_speed = []
    
    while True:
        for event in pg.event.get():
            if event.type == QUIT:
                pg.quit()
                
        no_still_alive = 0

        for i, car in enumerate(cars):
            output = networks[i].activate(car.get_neat_data())
            choice = output.index(max(output))

            car.move(choice)                
                
            if car.is_alive():
                no_still_alive += 1
                car.update()
                genomes[i][1].fitness = car.get_reward()
                
                avg_speed.append(car.speed)
                

        if no_still_alive == 0 or time.time() - timer > MAX_TIME:
            break

        screen.blit(my_track.game_map, (0, 0))
        for car in cars:
            if car.is_alive():
                car.draw(screen)

        text_gen = font_generation.render("Generation: " + str(current_generation), True, (0, 0, 0))
        text_gen_rect = text_gen.get_rect()
        text_gen_rect.center = (current_map['x_text'], current_map['y_text'])
        screen.blit(text_gen, text_gen_rect)

        text_alive = font_alive.render("Alive: " + str(no_still_alive), True, (0, 0, 0))
        text_alive_rect = text_alive.get_rect()
        text_alive_rect.center = (current_map['x_text'], current_map['y_text'] + current_map['distance_text'])
        screen.blit(text_alive, text_alive_rect)

        pg.display.flip()
        clock.tick(60)
        
    my_car = cars[0]
    
    avg_speed = sum(avg_speed) / len(avg_speed) if len(avg_speed) > 0 else 0
    
    return [my_car.distance, avg_speed]

    # pg.quit()

In [5]:
for i in range(len(maps)):
    current_map = maps[i]
     
    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, 'alg_neat/neat.config')
    current_generation = 0
        
    best_genome = None
    with open(f'alg_neat/best_genome_{maps[i]["name"]}.pkl', 'rb') as input:
        best_genome = pickle.load(input).best_genome
        
    analytics = run_simulation_competition([(1, best_genome)], config, speed = 20, full_screen = True)
    
    print(f'Analytics for {maps[i]["name"]}: {analytics}')

pg.quit()

Analytics for track01: [27945, 9.434503713706954]
Analytics for track02: [111359, 29.838960342979636]


In [6]:
for i in range(len(maps)):
    current_map = maps[i]
    for j in range(len(maps)):
        if i == j:
            continue
        
        config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, 'alg_neat/neat.config')
        current_generation = 0
            
        best_genome = None
        with open(f'alg_neat/best_genome_{maps[j]["name"]}.pkl', 'rb') as input:
            best_genome = pickle.load(input).best_genome
            
        analytics = run_simulation_competition([(1, best_genome)], config, speed = 20, full_screen = True)
        
        print(f'Analytics for {maps[i]["name"]} using best_genome_{maps[j]["name"]}: {analytics}')

pg.quit()

Analytics for track01 using best_genome_track02: [1278, 29.045454545454547]
Analytics for track02 using best_genome_track01: [1552, 17.839080459770116]
